In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1007763,2021-02-17,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1007764,2021-02-18,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1007765,2021-02-19,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1007766,2021-02-20,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
27395,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27397,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27399,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27401,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27403,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
624793,2021-02-17,Arkansas,Arkansas,5001,1933,38.00,5000,Arkansas,AR,Arkansas,State,3017804
624795,2021-02-18,Arkansas,Arkansas,5001,1939,38.00,5000,Arkansas,AR,Arkansas,State,3017804
624797,2021-02-19,Arkansas,Arkansas,5001,1939,38.00,5000,Arkansas,AR,Arkansas,State,3017804
624799,2021-02-20,Arkansas,Arkansas,5001,1941,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1007763,2021-02-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1007764,2021-02-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1007765,2021-02-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1007766,2021-02-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-21') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
397,2021-02-21,Snohomish,Washington,30297,526.00,53061,WA,County,822083,3685.39,63.98
792,2021-02-21,Cook,Illinois,469935,9809.00,17031,IL,County,5150233,9124.54,190.46
1186,2021-02-21,Orange,California,259305,3810.00,6059,CA,County,3175692,8165.31,119.97
1579,2021-02-21,Maricopa,Arizona,504992,8835.00,4013,AZ,County,4485414,11258.54,196.97
1972,2021-02-21,Los Angeles,California,1181060,19885.00,6037,CA,County,10039107,11764.59,198.08
...,...,...,...,...,...,...,...,...,...,...,...
1007363,2021-02-21,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1007495,2021-02-21,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
1007596,2021-02-21,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1007693,2021-02-21,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
397,2021-02-21,Snohomish,Washington,30297,526.00,53061,WA,County,822083,3685.39,63.98,63.98,3685.39
792,2021-02-21,Cook,Illinois,469935,9809.00,17031,IL,County,5150233,9124.54,190.46,190.46,9124.54
1186,2021-02-21,Orange,California,259305,3810.00,6059,CA,County,3175692,8165.31,119.97,119.97,8165.31
1579,2021-02-21,Maricopa,Arizona,504992,8835.00,4013,AZ,County,4485414,11258.54,196.97,196.97,11258.54
1972,2021-02-21,Los Angeles,California,1181060,19885.00,6037,CA,County,10039107,11764.59,198.08,198.08,11764.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007363,2021-02-21,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1007495,2021-02-21,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
1007596,2021-02-21,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1007693,2021-02-21,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
710673,2021-02-21,Clarke,Alabama,3419,48.00,1025,AL,County,23622,14473.80,203.20,203.20,14473.80,1
653178,2021-02-21,Hale,Alabama,2060,64.00,1065,AL,County,14651,14060.47,436.83,436.83,14060.47,2
511581,2021-02-21,Lowndes,Alabama,1298,46.00,1085,AL,County,9726,13345.67,472.96,472.96,13345.67,3
453805,2021-02-21,Etowah,Alabama,13056,307.00,1055,AL,County,102268,12766.46,300.19,300.19,12766.46,4
511914,2021-02-21,Marengo,Alabama,2381,54.00,1091,AL,County,18863,12622.59,286.27,286.27,12622.59,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958632,2021-02-21,Platte,Wyoming,580,11.00,56031,WY,County,8393,6910.52,131.06,131.06,6910.52,19
623811,2021-02-21,Sublette,Wyoming,671,7.00,56035,WY,County,9831,6825.35,71.20,71.20,6825.35,20
861001,2021-02-21,Niobrara,Wyoming,148,2.00,56027,WY,County,2356,6281.83,84.89,84.89,6281.83,21
774245,2021-02-21,Lincoln,Wyoming,1210,11.00,56023,WY,County,19830,6101.87,55.47,55.47,6101.87,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
511581,2021-02-21,Lowndes,Alabama,1298,46.00,1085,AL,County,9726,13345.67,472.96,472.96,13345.67,3,1
653178,2021-02-21,Hale,Alabama,2060,64.00,1065,AL,County,14651,14060.47,436.83,436.83,14060.47,2,2
452469,2021-02-21,Clay,Alabama,1419,54.00,1027,AL,County,13235,10721.57,408.01,408.01,10721.57,22,3
551538,2021-02-21,Greene,Alabama,864,32.00,1063,AL,County,8111,10652.20,394.53,394.53,10652.20,24,4
218178,2021-02-21,Walker,Alabama,6447,240.00,1127,AL,County,63521,10149.40,377.83,377.83,10149.40,32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550209,2021-02-21,Hot Springs,Wyoming,359,3.00,56017,WY,County,4413,8135.06,67.98,67.98,8135.06,16,19
755070,2021-02-21,Uinta,Wyoming,2027,12.00,56041,WY,County,20226,10021.75,59.33,59.33,10021.75,5,20
774245,2021-02-21,Lincoln,Wyoming,1210,11.00,56023,WY,County,19830,6101.87,55.47,55.47,6101.87,22,21
217158,2021-02-21,Teton,Wyoming,3326,9.00,56039,WY,County,23464,14174.91,38.36,38.36,14174.91,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.98,3685.39,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.98,3685.39,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.98,3685.39,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.98,3685.39,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.98,3685.39,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005383,2021-02-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1005384,2021-02-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1005385,2021-02-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1005386,2021-02-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
707967,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,203.20,14473.80,45,1,2.00
707968,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,203.20,14473.80,45,1,0.00
707969,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,203.20,14473.80,45,1,1.00
707970,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,203.20,14473.80,45,1,4.00
707971,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,203.20,14473.80,45,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787089,2021-02-17,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
787090,2021-02-18,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
787091,2021-02-19,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
787092,2021-02-20,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
509531,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13345.67,1,3,1.00,0.00
509532,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13345.67,1,3,0.00,0.00
509533,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13345.67,1,3,0.00,0.00
509534,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13345.67,1,3,0.00,0.00
509535,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,472.96,13345.67,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507861,2021-02-17,Albany,Wyoming,3871,11.00,56001,WY,County,38880,9956.28,28.29,28.29,9997.43,23,6,4.00,0.00
507862,2021-02-18,Albany,Wyoming,3871,11.00,56001,WY,County,38880,9956.28,28.29,28.29,9997.43,23,6,0.00,0.00
507863,2021-02-19,Albany,Wyoming,3880,11.00,56001,WY,County,38880,9979.42,28.29,28.29,9997.43,23,6,9.00,0.00
507864,2021-02-20,Albany,Wyoming,3880,11.00,56001,WY,County,38880,9979.42,28.29,28.29,9997.43,23,6,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-21') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
260306,2021-02-21,Imperial,California,26840,583.00,6025,CA,County,181215,14811.14,321.72,321.72,14811.14,1,2,0.00,0.00
1972,2021-02-21,Los Angeles,California,1181060,19885.00,6037,CA,County,10039107,11764.59,198.08,198.08,11764.59,2,5,1427.00,92.00
455093,2021-02-21,Inyo,California,1226,34.00,6027,CA,County,18039,6796.39,188.48,188.48,6796.39,3,30,14.00,0.00
58209,2021-02-21,Stanislaus,California,55191,926.00,6099,CA,County,550660,10022.70,168.16,168.16,10022.70,4,10,110.00,0.00
72126,2021-02-21,Tulare,California,47399,720.00,6107,CA,County,466195,10167.20,154.44,154.44,10167.20,5,9,0.00,0.00
27042,2021-02-21,Riverside,California,287534,3633.00,6065,CA,County,2470546,11638.48,147.05,147.05,11638.48,6,6,0.00,0.00
551480,2021-02-21,Kings,California,21857,215.00,6031,CA,County,152940,14291.23,140.58,140.58,14291.23,7,3,4.00,1.00
412565,2021-02-21,Merced,California,28489,388.00,6047,CA,County,277680,10259.65,139.73,139.73,10259.65,8,8,0.00,0.00
26338,2021-02-21,Fresno,California,94234,1367.00,6019,CA,County,999101,9431.88,136.82,136.82,9431.88,9,14,261.00,0.00
46345,2021-02-21,San Joaquin,California,65891,1026.00,6077,CA,County,762148,8645.43,134.62,134.62,8645.43,10,18,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
958183,2021-02-21,Lassen,California,5558,19.00,6035,CA,County,30573,18179.44,62.15,62.15,18179.44,36,1,0.00,0.00
260306,2021-02-21,Imperial,California,26840,583.00,6025,CA,County,181215,14811.14,321.72,321.72,14811.14,1,2,0.00,0.00
551480,2021-02-21,Kings,California,21857,215.00,6031,CA,County,152940,14291.23,140.58,140.58,14291.23,7,3,4.00,1.00
129884,2021-02-21,San Bernardino,California,284639,2612.00,6071,CA,County,2180085,13056.33,119.81,119.81,13056.33,13,4,359.00,55.00
1972,2021-02-21,Los Angeles,California,1181060,19885.00,6037,CA,County,10039107,11764.59,198.08,198.08,11764.59,2,5,1427.00,92.00
27042,2021-02-21,Riverside,California,287534,3633.00,6065,CA,County,2470546,11638.48,147.05,147.05,11638.48,6,6,0.00,0.00
163197,2021-02-21,Kern,California,101738,773.00,6029,CA,County,900202,11301.69,85.87,85.87,11301.69,24,7,125.00,12.00
412565,2021-02-21,Merced,California,28489,388.00,6047,CA,County,277680,10259.65,139.73,139.73,10259.65,8,8,0.00,0.00
72126,2021-02-21,Tulare,California,47399,720.00,6107,CA,County,466195,10167.20,154.44,154.44,10167.20,5,9,0.00,0.00
58209,2021-02-21,Stanislaus,California,55191,926.00,6099,CA,County,550660,10022.70,168.16,168.16,10022.70,4,10,110.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
260306,2021-02-21,Imperial,California,26840,583.00,6025,CA,County,181215,14811.14,321.72,321.72,14811.14,1,2,0.00,0.00
1972,2021-02-21,Los Angeles,California,1181060,19885.00,6037,CA,County,10039107,11764.59,198.08,198.08,11764.59,2,5,1427.00,92.00
455093,2021-02-21,Inyo,California,1226,34.00,6027,CA,County,18039,6796.39,188.48,188.48,6796.39,3,30,14.00,0.00
58209,2021-02-21,Stanislaus,California,55191,926.00,6099,CA,County,550660,10022.70,168.16,168.16,10022.70,4,10,110.00,0.00
72126,2021-02-21,Tulare,California,47399,720.00,6107,CA,County,466195,10167.20,154.44,154.44,10167.20,5,9,0.00,0.00
27042,2021-02-21,Riverside,California,287534,3633.00,6065,CA,County,2470546,11638.48,147.05,147.05,11638.48,6,6,0.00,0.00
551480,2021-02-21,Kings,California,21857,215.00,6031,CA,County,152940,14291.23,140.58,140.58,14291.23,7,3,4.00,1.00
412565,2021-02-21,Merced,California,28489,388.00,6047,CA,County,277680,10259.65,139.73,139.73,10259.65,8,8,0.00,0.00
26338,2021-02-21,Fresno,California,94234,1367.00,6019,CA,County,999101,9431.88,136.82,136.82,9431.88,9,14,261.00,0.00
46345,2021-02-21,San Joaquin,California,65891,1026.00,6077,CA,County,762148,8645.43,134.62,134.62,8645.43,10,18,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,321.72,14811.14,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3402,02/17/21,Lassen,5546,19.00,18140.19,62.15,62.15,18179.44,36,1,0.00,0.00
3403,02/18/21,Lassen,5558,19.00,18179.44,62.15,62.15,18179.44,36,1,12.00,0.00
3404,02/19/21,Lassen,5558,19.00,18179.44,62.15,62.15,18179.44,36,1,0.00,0.00
3405,02/20/21,Lassen,5558,19.00,18179.44,62.15,62.15,18179.44,36,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,321.72,14811.14,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,321.72,14811.14,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3476,02/17/21,San Joaquin,65659,1018.00,8614.99,133.57,134.62,8645.43,10,18,300.00,0.00
3477,02/18/21,San Joaquin,65705,1021.00,8621.03,133.96,134.62,8645.43,10,18,46.00,3.00
3478,02/19/21,San Joaquin,65891,1026.00,8645.43,134.62,134.62,8645.43,10,18,186.00,5.00
3479,02/20/21,San Joaquin,65891,1026.00,8645.43,134.62,134.62,8645.43,10,18,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)